## Description
Main python script used to assess climate characteristics and adaptation measures in Pisor et al. (One Earth)<br>
Sricpts by [Danielle Touma](danielletouma.com)<br>
Run script in Jupyter Notebook or platform with *.ipynb capabilities

### Dependencies
Run calculate_percipitation_percentiles.ipynb first to download precipitation data and calculate percentile thresholds.

### External functions
[find_runs.py](https://gist.github.com/alimanfoo/c5977e87111abe8127453b21204c1065)

## Main script

### Import packages/libraries

In [ ]:
# analysis packages
import numpy as np
import pandas as pd

#netcdf packages
import netCDF4
from netCDF4 import Dataset 
import xarray as xr

# plotting pacakges
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.lines import Line2D
from matplotlib.colors import BoundaryNorm
import matplotlib.ticker as mticker
import matplotlib.patheffects as pe
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import MaxNLocator
import seaborn as sns

# other packages
from calendar import monthrange
import regionmask #using my-npl-ml environment
import geopandas as gp

### Basic set up

Initialize directories where you can find the precipitation dataset (dir_month),
the threshold dataset (dir_pxx), shapefile (dir_shp), and remittance data (dir_rem), and
where figures should be output (dir_fig).

In [ ]:
dir_pxx = "/glade/scratch/detouma/adaptation/CHIRPS_monthly_pr/"
dir_month = "/glade/scratch/detouma/adaptation/CHIRPS_monthly_pr/"
dir_fig = "/glade/scratch/detouma/adaptation/figures/"
dir_shp =  "/glade/scratch/detouma/adaptation/shapefiles/"
dir_rem = "/glade/scratch/detouma/adaptation/data/"

Initialize variables that describe the time dimension of your precipitation dataset.

In [ ]:
year0 = 1981
year1 = 2021
nyears = year1-year0+1
years = np.arange(year0,year1+1,1)
ntime = nyears*12
survey_year = 2009

If your precipitation dataset does not have a date variable, create one for ease of indexing.
Otherwise, read in the date variable from your precipitation dataset

In [ ]:
date = np.zeros(ntime,dtype=int)
date_frac = np.zeros(ntime,dtype=float)
for yy in range(0,nyears,1):
    date[yy*12:(yy+1)*12] = (year0+yy)*100+np.arange(1,13,1)
    date_frac[yy*12:(yy+1)*12] = (year0+yy)+np.arange(0,12,1)/12

yyyy = np.floor(date_frac)

### Reading in climate variables and thresholds

Read in file with pixel-relevant percentile thresholds

In [ ]:
file_pxx = dir_pxx+'CHIRPS_'+str(year0)+'-'+str(year1)+'_pr_monthly_percentiles_africa.nc'
xr_pxx = xr.open_dataset(file_pxx)
pxx_list = np.array(xr_pxx['pxx'])
n_pxx = len(pxx_list)
lat = xr_pxx['lat']
lon = xr_pxx['lon']
nlat = len(lat)
nlon = len(lon)
pr_pxx = np.array(xr_pxx['pr'])

Read in monthly precipitation dataset and subset for lats/lons if needed

In [ ]:
file_pr = dir_month+'chirps_monthly_dataset.nc'
xr_pr = Dataset(file_pr,'r')
lat0 = xr_pr['latitude'][:]
lon0 = xr_pr['longitude'][:]
lat_inds = np.where((lat0>=-40)&(lat0<=40))[0]
lon_inds = np.where((lon0>=-20)&(lon0<=60))[0]
pr = xr_pr['precip'][0:ntime,lat_inds,lon_inds]
pr_month = np.reshape(pr,newshape=(int(ntime/12),12,nlat,nlon))

### FIGURE 1

Calculate percentiles for plotting Figure 1

In [ ]:
pr_percentile_month = np.full(fill_value=-999.0,shape=pr_month.shape)
for mm in range(0,12,1):
    for ii in range(1000,1100,1):
        for jj in range(300,400,1):
            for yy in range(0,nyears,1):
                n_below = np.sum(pr_month[:,mm,ii,jj]<=pr_month[yy,mm,ii,jj])
                pr_percentile_month[yy,mm,ii,jj] = n_below/nyears*100
                
pr_percentile = np.reshape(pr_percentile_month,newshape=(ntime,nlat,nlon))

Set up example parameters and process climate data

In [ ]:
ex_year0 = 1991 #start year of period
ex_year1 = 2009 #end year of period
ex_year_inds = np.where((yyyy>=ex_year0)&(yyyy<=ex_year1))[0]
ex_lat_ind = 1050
ex_lon_ind = 350

nyears_window = 5

pr_perc_example_mov_avg_all = pd.Series(pr_percentile[:,ex_lat_ind,ex_lon_ind]).rolling(window=nyears_window*12, center = True).mean()
pr_perc_example_mov_std_all = pd.Series(pr_percentile[:,ex_lat_ind,ex_lon_ind]).rolling(window=nyears_window*12, center = True).std()
pr_perc_example_mov_avg = pr_perc_example_mov_avg_all[ex_year_inds]
pr_perc_example_mov_std = pr_perc_example_mov_std_all[ex_year_inds]
pr_perc_example_mov_lo = pr_perc_example_mov_avg - pr_perc_example_mov_std
pr_perc_example_mov_hi = pr_perc_example_mov_avg + pr_perc_example_mov_std

pr_perc_example = pr_percentile[ex_year_inds,ex_lat_ind,ex_lon_ind]

date_frac_example = date_frac[ex_year_inds]

drought_points= np.where((pr_perc_example<=10))[0]
wet_points = np.where(pr_perc_example>=90)[0]


Plot Figure 1 - additional labeling was implemented using Adobe Illustrator

In [ ]:
fig1 = plt.figure(figsize=(12,4))
plt.plot(date_frac_example,pr_perc_example,color='grey',lw=0.75) # monthly time series
plt.plot(date_frac_example[drought_points], pr_perc_example[drought_points], 'ro') # drought months
plt.plot(date_frac_example[wet_points], pr_perc_example[wet_points], 'bo') # wet months
plt.fill_between(date_frac_example,pr_perc_example_mov_lo,pr_perc_example_mov_hi,color='gray',alpha=0.2) # 5 year moving st. dev.
plt.plot(date_frac_example,pr_perc_example_mov_avg,color='gray',lw=3) # 5 year moving mean

plt.xticks(np.arange(ex_year0,ex_year1+2,1),np.arange(ex_year0,ex_year1+2,1)) # fix tick labels
plt.axhline(10,color='red') # 10th percentile
plt.axhline(90, color='blue') # 90the percentiles

plt.title('Percentiles of monthly precipitation for '+str(np.round(lat[1050].data,2))+'N, '+str(np.round(lon[350].data,2))+'E')
plt.savefig(dir_fig+'percentiles_timeseries_example_3_raw.pdf')

### FIGURE 3

#### Read in country shapefiles

Read in shape files for administrative regions in Burkina Faso and zoom into Burkina Faso

In [ ]:
adm = 2 # choose level of administrative regions
adm_files = ['BFA_adm/BFA_adm0.shp','BFA_adm/BFA_adm1.shp','BFA_adm/BFA_adm2.shp','BFA_adm/BFA_adm3.shp']
gp_file_adm = gp.read_file(dir_shp+adm_files[adm])

adm_mask = regionmask.mask_geopandas(gp_file_adm, lon, lat)
adm_regions = regionmask.from_geopandas(gp_file_adm, 
                                        names="NAME_2", abbrevs="_from_name", name="Burkina_Faso_Adm_"+str(adm))
n_adms = len(adm_regions)

# choose lat/lon limits for plotting and analysis purposes. This helps reduce computational time.
burkina_lon_lims = [-6, 2.75]
burkina_lat_lims = [8.9, 15.5]

# find indices of limits - will improve code using xarray package. Coming soon.
lat_floor = np.floor(lat)
lat_ceil = np.ceil(lat)
lon_floor = np.floor(lon)
lon_ceil = np.ceil(lon)
lat_ind0 = np.where(np.floor(burkina_lat_lims[0])==lat_floor)[0][0]
lat_ind1 = np.where(np.ceil(burkina_lat_lims[1])==lat_ceil)[0][-1]
lon_ind0 = np.where(np.floor(burkina_lon_lims[0])==lon_floor)[0][0]
lon_ind1 = np.where(np.ceil(burkina_lon_lims[1])==lon_ceil)[0][-1]


#### Climate data processing

Create a binary variable storing whether or not a pixel is below a certain drought threshold (1 = drought, 0 = no drought]). We use the 10th percentile (xx=2) in this example.

In [ ]:
xx = 2 # index of pxx_list where the 10th percentile located.

pr_drought_bin = np.zeros((int(ntime/12),12,nlat,nlon), dtype=int) # nyears x nmonths x nlat x nlon integer dataset filled with zeros
for yy in range(0,nyears,1):
    #print(years[yy], end=';') # uncomment this line to print progress through loop
    # find indices of year[yy] that are undergoing drought. drought_inds.shape = (nmonths x nlat x nlon)
    drought_inds = np.where((pr_month[yy,:,:,:]<=pr_pxx[xx,:,:,:]) & (pr_month[yy,:,:,:]>=0))
    # set those indices to 1
    pr_drought_bin[yy,drought_inds[0],drought_inds[1],drought_inds[2]] = 1
# end of yearly loop

#reshape to make one time series for each lat/lon (3d array)
pr_drought_bin_3d = np.reshape(pr_drought_bin,newshape=(ntime,nlat,nlon))

Create a variable storing the *departure* (*i<sub>1</sub>* and *i<sub>2</sub>* in Figure 1) as the difference between the precipitation and the drought threshold for each pixel and month that is in drought. 

In [ ]:
# nyears x nmonths x nlat x nlon float dataset filled with a large positive number (1e36). For a flood threshold, use a large negative number.
pr_drought_departure = np.full(fill_value=1e36, shape = (int(ntime/12),12,nlat,nlon), dtype=float)
for yy in range(0,nyears,1):
    # print(years[yy], end=';') # uncomment this line to print progress through loop
    # find indices of year[yy] that are undergoing drought. drought_inds.shape = (nmonths x nlat x nlon)
    drought_inds = np.where((pr_month[yy,:,:,:]<=pr_pxx[xx,:,:,:]) & (pr_month[yy,:,:,:]>=0))
    # calculate precipitation amount below threshold for each drought month/location
    pr_drought_departure[yy,drought_inds[0],drought_inds[1],drought_inds[2]] = pr_month[yy,drought_inds[0],drought_inds[1],drought_inds[2]] - \
                                                                           pr_pxx[xx,drought_inds[0],drought_inds[1],drought_inds[2]]
# end of yearly loop

# mask out months/locations where no drought occurred. i.e., where pr_drought_intensity is still equal to 1e36. 
pr_drought_departure = np.ma.masked_array(pr_drought_departure, mask=(pr_drought_departure==1e36))

Find one or more consecutive months with drought. drought_onsets = 1 when a drought starts over each grid point. drought_lengths = corresponding drought duration of that drought onset

In [ ]:
# First, find locations where no precipitation occurs at all.
# Create a mask and set = 1 in these locations.
norain_inds = np.where(np.sum(pr,axis=0)==0)
norain_mask = np.zeros((nlat,nlon),dtype=int)
norain_mask[norain_inds[0],norain_inds[1]] = 1
norain_mask_3d = np.broadcast_to(norain_mask,pr.shape)
norain_mask_4d = np.broadcast_to(norain_mask,pr_month.shape)

In [ ]:
from find_runs import * # function found in find_runs.py

drought_onsets = np.zeros(pr_drought_bin_3d.shape,dtype=float)
drought_lengths = np.zeros(pr_drought_bin_3d.shape,dtype=float)

for ii in range(lat_ind0,lat_ind1+1,1): # loop through lats
    # print(str(ii+1)+'/'+str(nlat), end=';') # uncomment this line to print progress through loop
    for jj in range(lon_ind0,lon_ind1+1,1): # loop through lons
        # check if there are any drought months and if the grid point has no rain at all
        if ((np.sum(pr_drought_bin_3d[:,ii,jj])>0)&(norain_mask[ii,jj]==0)):
            v, s, l = find_runs(pr_drought_bin_3d[:,ii,jj])
            # v = value of run. in our case, 1 = drought, 0 = no drought
            # s = index of run starts
            s_droughts = s[np.where(v==1)] # find indices of drought starts (i.e., v == 1)
            # l = length of runs
            l_droughts = l[np.where(v==1)] # find lengths of drought runs (i.e., v == 1)
            drought_onsets[s_droughts,ii,jj] = 1
            drought_lengths[s_droughts,ii,jj] = l_droughts
            
drought_lengths_4d = np.reshape(drought_lengths, newshape = (nyears,12,nlat,nlon))
drought_lengths_4d = np.ma.masked_equal(drought_lengths_4d,0)

drought_onsets_4d = np.reshape(drought_onsets, newshape = (nyears,12,nlat,nlon))

For each drought run/event, find the magnitude, severity, and intensity of that drought using the *departure* (*i<sub>1</sub>* and *i<sub>2</sub>* in Figure 1).

In [ ]:
drought_severity = np.full(fill_value = 999.0, shape=pr_drought_bin_3d.shape,dtype=float) 
drought_magnitude = np.full(fill_value = 999.0, shape=pr_drought_bin_3d.shape,dtype=float)
drought_intensity = np.full(fill_value = 999.0, shape=pr_drought_bin_3d.shape,dtype=float)

drought_departure_3d = np.reshape(pr_drought_departure,pr_drought_bin_3d.shape)

for ii in range(lat_ind0,lat_ind1+1,1): # loop through lats
    # print(str(ii+1)+'/'+str(nlat), end=';') # uncomment this line to print progress through loop
    for jj in range(lon_ind0,lon_ind1+1,1): # loop through lons
        start_inds = np.where(drought_onsets[:,ii,jj]==1)[0]
        if len(start_inds)>0:
            for ss in start_inds:
                drought_severity[ss,ii,jj] = np.ma.sum(drought_departure_3d[ss:ss+int(drought_lengths[ss,ii,jj]),ii,jj])
                drought_magnitude[ss,ii,jj] = np.ma.min(drought_departure_3d[ss:ss+int(drought_lengths[ss,ii,jj]),ii,jj])
                drought_intensity[ss,ii,jj] = np.ma.mean(drought_departure_3d[ss:ss+int(drought_lengths[ss,ii,jj]),ii,jj])

#mask any places where no drought occured
drought_severity = np.ma.masked_equal(drought_severity,999.0)
drought_magnitude = np.ma.masked_equal(drought_magnitude,999.0)
drought_intensity = np.ma.masked_equal(drought_intensity,999.0)

# reshape to a monthly array
drought_severity_4d = drought_severity.reshape((nyears,12,nlat,nlon))
drought_magnitude_4d = drought_magnitude.reshape((nyears,12,nlat,nlon))
drought_intensity_4d = drought_intensity.reshape((nyears,12,nlat,nlon))


For each pixel, find
  * annual drought frequency,
  * annual average drought intensity,
  * annual drought duration,
  * annual drought severity, and
  * annual drought magnitude.

In [ ]:
yearly_drought_frequency = np.ma.sum(drought_onsets_4d,axis=1)
yearly_drought_intensity = np.ma.mean(drought_intensity_4d,axis=1)
yearly_drought_duration = np.ma.mean(drought_lengths_4d, axis=1)
yearly_drought_severity = np.ma.mean(drought_severity_4d, axis=1)
yearly_drought_magnitude = np.ma.mean(drought_magnitude_4d, axis=1)

For each pixel, find dispersion using metric from [Mailier et al. (2006)](https://journals.ametsoc.org/view/journals/mwre/134/8/mwr3160.1.xml)

In [ ]:
mean_drought_frequency = np.ma.mean(yearly_drought_frequency,axis=0)
var_drought_frequency = np.ma.var(yearly_drought_frequency,axis=0)
all_drought_dispersion = var_drought_frequency/mean_drought_frequency-1

Select period(s) of interest to calculate frequency, duration, intensity, magnitude, severity, and dispersion and plot. This is the data that is plotted in Figure 3.

In [ ]:
period_year0 = [1991, 2005] #start year of period
period_year1 = [2009, 2009] #end year of period
nperiods = len(period_year0) # number of periods

# create arrays to store period averages
period_drought_intensity = np.full(fill_value=-999.0,shape=(nperiods,nlat,nlon))
period_drought_duration = np.full(fill_value=-999.0,shape=(nperiods,nlat,nlon))
period_drought_frequency = np.full(fill_value=-999.0,shape=(nperiods,nlat,nlon))
period_drought_severity = np.full(fill_value=-999.0,shape=(nperiods,nlat,nlon))
period_drought_magnitude = np.full(fill_value=-999.0,shape=(nperiods,nlat,nlon))
period_drought_dispersion = np.full(fill_value=-999.0,shape=(nperiods,nlat,nlon))

for pp in range(0,nperiods,1):
    period_year_inds = np.where((years>=period_year0[pp])&(years<=period_year1[pp]))[0]
    period_drought_intensity[pp,:,:] = np.ma.mean(yearly_drought_intensity[period_year_inds,:,:], axis=0)
    period_drought_duration[pp,:,:] = np.ma.mean(yearly_drought_duration[period_year_inds,:,:], axis=0)
    period_drought_frequency[pp,:,:] = np.ma.mean(yearly_drought_frequency[period_year_inds,:,:], axis=0)
    period_drought_severity[pp,:,:] = np.ma.mean(yearly_drought_severity[period_year_inds,:,:], axis=0)
    period_drought_magnitude[pp,:,:] = np.ma.mean(yearly_drought_magnitude[period_year_inds,:,:], axis=0)
    period_drought_frequency_var = np.ma.var(yearly_drought_frequency[period_year_inds,:,:], axis=0)
    period_drought_dispersion[pp,:,:] = period_drought_frequency_var/period_drought_frequency[pp,:,:] - 1

#### Remittance data processing

Read in remittance data which will be plotted on top of the climate characteristics. For our example shown in Figure 3, we downloaded [Migration and Remittance Survey data from the World Bank](https://microdata.worldbank.org/index.php/catalog/mrs/), which we then subset for Burkina Faso.

In [ ]:
meso_df = pd.read_csv(dir_rem+'remittance data_meso.csv') 
burkina_df = meso_df[meso_df['country']=='burkina'] #subset burkina faso

Extract number of total remittances for each region based on shape file of administrative regions and the coordinates of the remittance data. In this dataset, remittances are coded as 1 (yes) or 0 (no).

In [ ]:
# create a geodataframe for the remittance data with lat/lons
rem_loc = gp.GeoDataFrame(burkina_df, geometry=gp.points_from_xy(burkina_df.long,burkina_df.lat)) 
# add a 'region' column and find region name based on lat/lons
rem_loc['region'] = ''
for rr in range(gp_file_adm.shape[0]):
    pip = rem_loc.within(gp_file_adm.loc[rr, 'geometry'])
    if pip.sum() > 0:
        rem_loc.loc[pip, 'region'] = gp_file_adm.loc[rr, 'NAME_'+str(adm)]

# count the total number of remittances for each region (=1) and create a dataframe
n_remit_df = rem_loc.groupby(['region','remit']).count().reset_index()[['region','remit','house']]
# find the percent of remittances by dividing the total number of questionaires ('house') in each region
n_remit_df['percent'] = n_remit_df['house']/n_remit_df.groupby('region')['house'].transform('sum')*100

# find the regions and number of regions represented in the remittance dataset
rem_regions = rem_loc['region'].unique()
n_rem_regions = len(rem_regions)

For plotting purpose, find a "representative point" at which the pie charts of remittance data will be plotted for each region.

In [ ]:
gp_file_adm['x_rep'] = gp_file_adm.representative_point().x
gp_file_adm['y_rep'] = gp_file_adm.representative_point().y

#### Plotting Figure 3

Set plotting features for the climate characteristics

In [ ]:
# text formatting for the regionmask country border plotting (basically, do not show any text)
text_kws = dict(bbox=dict(color="none"),color="none")
# line formatting for regionmask country border plotting
line_kws=dict(color='black',linewidth=1.5)

# contour levels, color map, and whether the color bar should extend for plotting - change depending on characteristic
dims  = ['frequency','duration','intensity','magnitude','severity','dispersion']
dims_long = ['Frequency (# of events)', 'Duration (months)','Intensity (mm/month)',
             'Magnitude (mm/month)', 'Severity (mm/drought)', 'Dispersion']
clevs = [np.arange(1,2.1,0.1),np.arange(1,2.1,0.1),np.arange(-12,0.1,1),
         np.arange(-12,0.1,1),np.arange(-12,0.1,1),np.arange(-1.4,1.5,0.1)]
cmap  = ['Reds','Purples','YlOrBr_r','YlOrBr_r','YlOrBr_r','RdYlBu_r']
extend = ['max','max','min','min','min','both'] #'min' #'both'

Plot frequency, duration, intensity, magnitude, severity, and dispersion for period of interest.

In [ ]:
pp = 0 # only plot for first period
for dd in range(0,len(dims),1): #loop through each climate dimension
    print(dims[dd])
    fig = plt.figure(figsize=(11,8.5)) # create a figure
    ax = plt.axes(projection=ccrs.PlateCarree()) # set the projection of the map
    dim_dd = eval('period_drought_'+dims[dd]) # variable name for the dimension
    levels = MaxNLocator(nbins=20).tick_values(np.min(dim_dd[pp,lat_ind0:lat_ind1,lon_ind0:lon_ind1]), # create color bar  contour levels
                                               np.max(dim_dd[pp,lat_ind0:lat_ind1,lon_ind0:lon_ind1]))
    cmap_d = plt.colormaps[cmap[dd]] # create color map for dimension
    norm = BoundaryNorm(levels, ncolors=cmap_d.N, clip=True) # normalization of the color map
    cs = ax.pcolormesh(lon,lat,eval('period_drought_'+dims[dd]+'[pp,:,:]'), # plot the climate dimension as a colormesh to see granularity of dataset
                       cmap=cmap_d, norm=norm,
                        transform = ccrs.PlateCarree())
    ax.add_feature(cfeature.BORDERS,edgecolor='grey') # add country borders
    ax.coastlines(color='grey') # add coastlines
    ax.set_extent([burkina_lon_lims[0], burkina_lon_lims[1], # zoom in the burkina faso
                   burkina_lat_lims[0], burkina_lat_lims[1]],
                  crs=ccrs.PlateCarree())
    ax.set_title(dims_long[dd] + ' ('+str(period_year0[pp])+'-'+str(period_year1[pp])+')', fontsize = 24) # title of figure
    adm_regions.plot_regions(ax=ax, add_label=False) # add administrative region borders
    cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height]) # add axis for color bar
    cbar = plt.colorbar(cs, cax=cax) # plot the color bar
    cbar.ax.tick_params(labelsize=20) # set the color bar ticks/levels
    for rr in range(0,n_rem_regions,1): #loop through regions with remittance data to plot remittance data as pie charts
        reg_name = rem_regions[rr] 
        lonr,latr = gp_file_adm['y_rep'][rr], gp_file_adm['x_rep'][rr] # set location of pie chart plot
        ax_sub = inset_axes(ax, width=0.55, height=0.55, loc=10, #add plot axis to for pie chart
                            bbox_to_anchor=(latr,lonr), bbox_transform=ax.transData)
        rr_perc_0 = n_remit_df[(n_remit_df['region']==reg_name)&(n_remit_df['remit']==0)]['percent'].values[0] # find "no" percent for region
        rr_perc_1 = n_remit_df[(n_remit_df['region']==reg_name)&(n_remit_df['remit']==1)]['percent'].values[0] # find "yes" percent for region
        rr_n_tot = np.sum(n_remit_df[(n_remit_df['region']==reg_name)]['house']) # find total number of remittances for region
        wedges, texts = ax_sub.pie([rr_perc_0, rr_perc_1], colors= ['white','grey'], startangle=90, # create pie chart
                    wedgeprops={'edgecolor': 'black','linewidth': 2, 'alpha':0.7})
        #t = ax_sub.text(x=0, y=1.5, s = str(rr_n_tot), ha='center', va='bottom', fontsize=14) # uncomment these lines if you would like to plot number of
        #t.set_bbox(dict(facecolor='white', alpha=0.7, edgecolor='none',boxstyle='square,pad=0.1')) # remittance data points
    ax.legend(wedges, ['no','yes'], title='Remittance', loc="upper left", title_fontsize=16, fontsize=20) # create legend for remittance data
    fig.savefig(dir_fig+'CHIRPS_drought_'+str(period_year0[pp])+'-'+str(period_year1[pp])+'_'+dims[dd]+'_'+str(pxx_list[xx])+ # save separate figure for each dimension
                '_percentile_events_w_'+str(survey_year)+'remittance_BurkinaFaso_admlevel'+str(adm)+'_levels.png')


#### Plotting inset scatterplots for Figure 3

Calculate regional averages of climate characteristics

In [ ]:
remit1_df = n_remit_df[n_remit_df['remit']==1].reset_index()
nregions = len(adm_regions.names)
dims_regions_avg_df = pd.DataFrame({'names':adm_regions.names*nperiods})
dims_regions_avg_df['period'] = ''
for dd in range(0,len(dims),1):
    # print(dims[dd]) # uncomment to print out loop progress
    dims_regions_avg_df[dims[dd]+'_avg'] = ''
    for pp in range(0,nperiods,1):
        dims_regions_avg_df.period[pp*nregions:(pp+1)*nregions] = pp
        dim_dd = eval('period_drought_'+dims[dd]+'[pp,:,:]')    
        for rr in range(0,n_adms,1):
            dims_regions_avg_df[dims[dd]+'_avg'][rr+(nregions*pp)] = np.ma.mean(dim_dd[adm_mask==rr])

Merge remittance dataframe with climate characteristics dataframe for regions

In [ ]:
perc1_df = n_remit_df[(n_remit_df['remit']==1)]
dims_remit_df = pd.merge(dims_regions_avg_df,perc1_df, right_on = 'region', left_on = 'names').drop(columns=['region','remit'])

Plot scatterplots for each climate characteristic against remittance percentages for different periods

In [ ]:
for dd in range(0,len(dims),1):
    fig, ax = plt.subplots(figsize=(5,5))
    sns.set_palette(sns.color_palette(['black','gray'])) # add colors for more periods
    #dims_remit_df.plot.scatter('percent',dims[dd]+'_avg', color='black',s=180, ax=ax)
    sns.scatterplot(data=dims_remit_df, x='percent', y=dims[dd]+'_avg',hue='period',s=180,style='period',ax=ax,legend=False)
    ax.set_ylabel('') #dims_long[dd], fontsize=24)
    ax.set_xlabel('') #'percent remittance', fontsize=24)
    ax.set_title(dims_long[dd],fontsize=20)
    ax.tick_params(axis='both', which='major', labelsize=18)
    fig.savefig(dir_fig+dims[dd]+'_v_remittance_perc_scatter_regions.png')